In [10]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#Training

# bertscore = load("bertscore") 
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-small-finetuned-text-simplification",cache_dir ="/scratch/aparna")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-small-finetuned-text-simplification",cache_dir ="/scratch/aparna")

In [12]:
#generate


max_l = 512
num_b = 10
num_sub_b =1

with open('../data/10/input.txt') as f:
    texts = f.readlines()
with open('../data/10/labels.txt') as f1:
    labels = f1.readlines()
dict1 = {}
for text in tqdm(texts):
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)
    dict1[text] = {'cand':[],'label':labels[texts.index(text)]}
    # generate text until the output length (which includes the context length) reaches 50
    beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
        no_repeat_ngram_size=3,
        num_return_sequences=num_b,
        top_k=4, top_p=0.95
        # return_dict_in_generate=
        #True,
    )
    # z = json.loads("../data/10/output1.json")
    # # appending the data
    # z.update(y)


    #print("====================================")
    for i, beam in enumerate(beam_outputs):
        dict1[text]['cand'].append(tokenizer.decode(beam, skip_special_tokens=True))
    
        #print("{}{}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))
# for x in range(10)
# with open('../data/10/output1.json', 'w') as fp:
#     json.dump(dict1, fp)


  0%|          | 0/33801 [00:00<?, ?it/s]

  0%|          | 120/33801 [01:42<8:00:37,  1.17it/s] 


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:21                                                                                   │
│                                                                                                  │
│ /home2/aparna/miniconda3/envs/svoice/lib/python3.7/site-packages/torch/autograd/grad_mode.py:27  │
│ in decorate_context                                                                              │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /home2/aparna/miniconda3/envs/svoice/lib/python3.7/site-packages/transformers/generation/utils.p │
│ y:1500 in generate                                                                               │
│                                                                                                  │
│   1497 │   │   │   │   output_scores=generation_config.output_scores,                            │
│   1498 │   │   │   │   return_dict_in_generate=generation_config.return_dict_in_generate,        │
│   1499 │   │   │   │   synced_gpus=synced_gpus,                                                  │
│ ❱ 1500 │   │   │   │   **model_kwargs,                                                           │
│   1501 │   │   │   )                                                                             │
│   1502 │   │                                                                                     │
│   1503 │   │   elif is_beam_sample_gen_mode:                                                     │
│                                                                                                  │
│ /home2/aparna/miniconda3/envs/svoice/lib/python3.7/site-packages/transformers/generation/utils.p │
│ y:2753 in beam_search                                                                            │
│                                                                                                  │
│   2750 │   │   │   │   **model_inputs,                                                           │
│   2751 │   │   │   │   return_dict=True,                                                         │
│   2752 │   │   │   │   output_attentions=output_attentions,                                      │
│ ❱ 2753 │   │   │   │   output_hidden_states=output_hidden_states,                                │
│   2754 │   │   │   )                                                                             │
│   2755 │   │   │                                                                                 │
│   2756 │   │   │   if synced_gpus and this_peer_finished:                                        │
│                                                                                                  │
│ /home2/aparna/miniconda3/envs/svoice/lib/python3.7/site-packages/torch/nn/modules/module.py:1194 │
│ in _call_impl                                                                                    │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.     

In [3]:
#generate
texts= ["Passionate friendship is a close but non-sexual relationship between friends","Electrical switches , fuses or circuit breakers were used to control , protect or isolate electrical equipment.","A hockey league belongs to the Canadian Junior Hockey League"]

for text in texts:
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_output = model.generate(inputs, max_length=512, 
        num_beams=10, top_k=120, top_p=0.95, early_stopping=True, num_return_sequences=10)
    print("====================================")
    for i, beam in enumerate(beam_output):
        print("{}: {}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))


0: Passionate friendship is a close but non-sexual relationship between friends.
1: Passionate friendship is a close relationship between friends.
2: Passionate friendship is a close but non-sexual relationship between friends and family.
3: Passionate friendship is a relationship between friends.
4: Passionate friendship is a close but nonsexual relationship between friends.
5: Passionate friendship is a close and non-sexual relationship between friends.
6: Passionate friendship is a close but non-sexual relationship between friends and friends.
7: Passionate friendship is a close friendship between friends.
8: Passionate friendship is a relationship between friends and family.
9: Passionate friendship is a close relationship between friends and family.
0: Electrical switches, fuses or circuit breakers were used to control, protect or isolate electrical equipment.
1: Electrical switches, fuses and circuit breakers were used to control, protect or isolate electrical equipment.
2: Elect

In [8]:
# %%
import numpy

from transformers import Trainer, TrainingArguments
from datasets import load_dataset
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from simpletransformers.t5 import T5Model, T5Args
from transformers import pipeline
#import train split
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import torch
import torch.nn as nn
#from google.transliteration import transliterate_word
#import klib
import os
#bleu score
from torchtext.data.metrics import bleu_score
import csv
colnames = ['source', 'target']
input_file = "train.tsv"
#df = pd.read_csv(input_file, sep="\t", quoting=csv.QUOTE_NONE, encoding='utf-8', header=None, names=colnames)
test = pd.read_csv("valid.tsv", sep="\t", quoting=csv.QUOTE_NONE, encoding='utf-8', header=None, names=colnames)
test = Dataset.from_pandas(test)
# %%
#tokenize
#tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")
tokenizer.add_special_tokens({'additional_special_tokens': ['<sep>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['<pad>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['<s>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['</s>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['<unk>']})

maxlen = 512
def tokenize_df(df):
    target = tokenizer(df['target'], padding='max_length', truncation=True, return_tensors="pt", max_length=maxlen)
    input = tokenizer(df['source'], padding='max_length', truncation=True, return_tensors="pt", max_length=maxlen)
    input_ids = input['input_ids']
    attention_mask = input['attention_mask']
    target_ids = target['input_ids']
    target_attention_mask = target['attention_mask']
    decoder_input_ids = target_ids.clone()
    #convert to tensors
    input_ids = torch.tensor(input_ids).squeeze()
    attention_mask = torch.tensor(attention_mask).squeeze()
    target_ids = torch.tensor(target_ids).squeeze()
    target_attention_mask = torch.tensor(target_attention_mask).squeeze()
   # decoder_input_ids = torch.tensor(decoder_input_ids)
    
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': target_ids,
        #'decoder_input_ids': decoder_input_ids,
        #'decoder_attention_mask': target_attention_mask
    }




# %%
# train = load_dataset('csv', data_files='train.csv')
# val = load_dataset('csv', data_files='val.csv')
# test = load_dataset('csv', data_files='test.csv')
# train = train.map(tokenize_df, batched=True, batch_size=128,remove_columns=['sentence','english_translation'])
# val = val.map(tokenize_df, batched=True, batch_size=128,remove_columns=['sentence','english_translation'])
test = test.map(tokenize_df, batched=True, batch_size=128,remove_columns=['source','target'])


# %%
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer


bleu_metric = evaluate.load("sacrebleu")

rouge_metric = evaluate.load("rouge")

def tokenize_sentence(arg):
    encoded_arg =tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

def metrics_func(eval_arg):
    preds, labels = eval_arg
    # Replace -100
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Convert id tokens to text
    text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Insert a line break (\n) in each sentence for ROUGE scoring
    # (Note : Please change this code, when you perform on other languages except for Japanese)
    text_preds = [(p if p.endswith(("!", "！", "?", "？", "。")) else p + "。") for p in text_preds]
    text_labels = [(l if l.endswith(("!", "！", "?", "？", "。")) else l + "。") for l in text_labels]
    sent_tokenizer_jp = RegexpTokenizer(u'[^!！?？。]*[!！?？。]')
    text_preds = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(p))) for p in text_preds]
    text_labels = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(l))) for l in text_labels]
    # compute ROUGE score with custom tokenization
    #blue score
    texts_bleu =[text.strip() for text in text_preds]
    labels_bleu = [[text.strip()] for text in text_labels]
    result = bleu_metric.compute(predictions=texts_bleu, references=labels_bleu)
    return rouge_metric.compute(
        predictions=text_preds,
        references=text_labels,
        tokenizer=tokenize_sentence
    ), result['score']

# %%
from torch.utils.data import DataLoader
#tokenizer = MT5Tokenizer.from_pretrained("./mt5")
def testing(model):
    metrics =[]
    sample_dataloader = DataLoader(
      test.with_format("torch"),
      collate_fn=DataCollatorForSeq2Seq(tokenizer, model=model),
      batch_size=5)
    for batch in sample_dataloader:
      with torch.no_grad():
        preds = model.generate(
          batch["input_ids"],
          num_beams=15,
          num_return_sequences=1,
          no_repeat_ngram_size=1,
          remove_invalid_values=True,
          max_length=128,
        )
      labels = batch["labels"]
      metric = metrics_func([preds, labels])
      metrics.append(metric)
    return metrics

def average_metric(metrics):
    rouge = 0
    rouge2 = 0
    rougeL = 0
    rougeLsum = 0
    bleu = 0
    for metric in metrics:
        rouge += metric[0]['rouge1']
        rouge2 += metric[0]['rouge2']
        rougeL += metric[0]['rougeL']
        rougeLsum += metric[0]['rougeLsum']
        bleu += metric[1]
    return rouge/len(metrics),rouge2/len(metrics),rougeL/len(metrics),rougeLsum/len(metrics),bleu/len(metrics)
      

# %%

metrics = testing(model)

# %%
print("t5_small")
scores = average_metric(metrics)
print("rouge:",scores[0])
print("rouge2:",scores[1])
print("rougeL:",scores[2])
print("rougeLsum:",scores[3])
print("bleu:",scores[4])


# %%
from torch.utils.data import DataLoader

# Predict with test data (first 5 rows)
sample_dataloader = DataLoader(
  test.with_format("torch"),
  collate_fn=DataCollatorForSeq2Seq(tokenizer, model=model),
  batch_size=5)
for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"],
      num_beams=15,
      num_return_sequences=1,
      no_repeat_ngram_size=1,
      remove_invalid_values=True,
      max_length=128,
    )
  labels = batch["labels"]
  inputs = batch["input_ids"]
  break

# Replace -100 (see above)
inputs = np.where(inputs != -100, inputs, tokenizer.pad_token_id)
labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

# Convert id tokens to text
text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
text_inputs = tokenizer.batch_decode(inputs, skip_special_tokens=True)
#print(bleu_score(list(text_labels.split()),list(text_preds.split())))

# Show result
print("***** Input's Text *****")
print(text_inputs[2])
print("***** summmary (True Value) *****")
print(text_labels[2])
print("***** summary (Generated Text) *****")
print(text_preds[2])

# %%
for i in range(5):
    print("***** Input's Text *****")
    print(text_inputs[i])
    print("***** summarry (True Value) *****")
    print(text_labels[i])
    print("***** summary (Generated Text) *****")
    print(text_preds[i])




Map:   0%|          | 0/20000 [00:00<?, ? examples/s]/tmp/ipykernel_20575/1962435399.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids).squeeze()
/tmp/ipykernel_20575/1962435399.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(attention_mask).squeeze()
/tmp/ipykernel_20575/1962435399.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_ids = torch.tensor(target_ids).squeeze()
/tmp/ipykernel_20575/1962435399.py:50: UserWarning: To copy construct from a tensor, i